# 1. Using the Database and Config Files

## Config Files

Coding your models from scratch is tedious and prone to error. BMSS2 uses config files to expedite the process using .ini files!

There are 3 kinds of config files used in BMSS2.


### Core Models
<b>Core models</b> are descriptions of model topologies and can be stored in the database. They contain only information on the variables involved in the model but not their actual values. An example of a simple model involving mRNA transcription and translation is shown below.
```
[system_type]
system_type = my_model

[states]
states = m, p

[parameters]
parameters = k_ind, synm, degm, synp, degp

[inputs]
inputs = ind

[equations]
equations =
	dm = synm*ind/(ind + k_ind) - degm*m
	dp = synp*m - degp*p

```


### Settings
Settings are information for carrying out simulation and other kinds of analyses. 

```
[my_model]
init = 
	m = [0, 5e-5],
	p = [0, 5e-6]

parameter_values = 
	k_ind = [1e-2],
	synm  = [1e-5],
	degm  = [0.015],
	synp  = [1e-2],
	degp  = [0.012],
	ind   = [0]

priors = 
	degm  = [0.015, 0.05],
	degp  = [0.012, 0.04]

parameter_bounds = 
	k_ind = [1e-3, 1],
	synm  = [1e-6, 1e-4],
	degm  = [0.01, 0.5],
	synp  = [1e-3, 1],
	degp  = [1e-3, 0.3],
	ind   = [0, 1]

units = 
	k_ind = % Arabinose,
	synm  = M/min,
	degm  = 1/min,
	synp  = M/min,
	degp  = 1/min,
	ind   = % Arabinose
```

Each core model requires its own file. Settings are however flexible and you can arrange them accrding to your needs. If you are analyzing multiple models in one go, we recommend putting the settings for all the models into a single file.

```
[model1]
some information...

[model2]
more information...


etc.
```

### Settings Templates

To speed up writing new settings, a settings templates can be used as a base for generating new settings with pre-filled values. Like <b>core models</b>, they can be stored in the database.

```
[my_settings]
system_type = my_model

init = 
	m = [0, 5e-5],
	p = [0, 5e-6]

parameter_values = 
	k_ind = [1e-2],
	synm  = [1e-5],
	degm  = [0.015],
	synp  = [1e-2],
	degp  = [0.012],
	ind   = [0]

priors = 
	degm  = [0.015, 0.05],
	degp  = [0.012, 0.04]

parameter_bounds = 
	k_ind = [1e-3, 1],
	synm  = [1e-6, 1e-4],
	degm  = [0.01, 0.5],
	synp  = [1e-3, 1],
	degp  = [1e-3, 0.3],
	ind   = [0, 1]

units = 
	k_ind = % Arabinose,
	synm  = M/min,
	degm  = 1/min,
	synp  = M/min,
	degp  = 1/min,
	ind   = % Arabinose
```

## Working with the Database

This section shows you how to use the config files for <b>core models</b> and <b>settings templates</b> in Python and how to add the information into the BMSS2 database. 

For now, the config data is stored in a file called ```my_model.ini``` and the .ini code is as follows:

```
[system_type]
system_type = my_model

[states]
states = m, p

[parameters]
parameters = k_ind, synm, degm, synp, degp

[inputs]
inputs = ind

[equations]
equations =
	dm = synm*ind/(ind + k_ind) - degm*m
	dp = synp*m - degp*p

[my_settings]
system_type = my_model

init = 
	m = [0, 5e-5],
	p = [0, 5e-6]

parameter_values = 
	k_ind = [1e-2],
	synm  = [1e-5],
	degm  = [0.015],
	synp  = [1e-2],
	degp  = [0.012],
	ind   = [0]

priors = 
	degm  = [0.015, 0.05],
	degp  = [0.012, 0.04]

parameter_bounds = 
	k_ind = [1e-3, 1],
	synm  = [1e-6, 1e-4],
	degm  = [0.01, 0.5],
	synp  = [1e-3, 1],
	degp  = [1e-3, 0.3],
	ind   = [0, 1]

units = 
	k_ind = % Arabinose,
	synm  = M/min,
	degm  = 1/min,
	synp  = M/min,
	degp  = 1/min,
	ind   = % Arabinose
```

### Importing Models into Python

In [1]:
import BMSS.models.model_handler as mh

Connected to MBase_models, UBase_models


Having imported the modules, we now read the .ini file to get a dictionary version of a core model.

In [2]:
filename     = 'my_model.ini'
core_model_1 = mh.from_config(filename)

for key, value in core_model_1.items():
    print(key)
    print(value)
    print()

id


system_type
my_model

states
['m', 'p']

parameters
['k_ind', 'synm', 'degm', 'synp', 'degp']

inputs
['ind']

equations
['dm = synm*ind/(ind + k_ind) - degm*m', 'dp = synp*m - degp*p']

ia


descriptions
{}



Note that some additional fields are generated by BMSS2. Just leave them be.

If you want to make your own core model, you can use the constructor in BMSS2 to ensure your data is formatted correctly.

In [3]:
core_model_2 = mh.make_core_model(system_type = core_model_1['system_type'],
                                  states      = core_model_1['states'],
                                  parameters  = core_model_1['parameters'],
                                  inputs      = core_model_1['inputs'],
                                  equations   = core_model_1['equations'],
                                  ia          = core_model_1['ia']
                                  )

As you work with more and more models, managing so many files can be a hassle. This is where the database comes in. Let's see the first few models inside the database.

In [4]:
mh.list_models()[:5]

['BMSS, ConstantInduction, Inducible',
 'BMSS, ConstantInduction, Inducible, MaturationTime',
 'BMSS, InducerDegradation, Inducible',
 'BMSS, InducerDegradation, Inducible, MaturationTime',
 'BMSS, DelayActivation, Inducible, ActiveTransport']

To search for a model, use the ```quick_search``` function.

In [5]:
mh.quick_search('BMSS, ConstantInduction, Inducible')

{'id': 'bmss1',
 'system_type': 'BMSS, ConstantInduction, Inducible',
 'states': ['m', 'p'],
 'parameters': ['n', 'k_ind', 'synm', 'degm', 'synp', 'degp'],
 'inputs': ['ind'],
 'equations': ['dm = synm*(ind**n)/(ind**n + k_ind**n) - degm*m',
  'dp = synp*m - degp*p'],
 'ia': '',
 'descriptions': {'Description': 'This model describes a simple gene expression for mRNA and protein driven by an inducible promoter with constant induction.',
  'Definition of states': '\nm: mRNA\np: protein',
  'Definition of parameters': '\nk_ind: half-activation constant\nsynm: mRNA synthesis rate\ndegm: mRNA degradation rate\nsynp: protein synthesis rate\ndegp: protein degradation rate/dilution rate\nn: hil coefficient',
  'Definition of inputs': '\nind: inducer level',
  'Reference': '\ntitle: An Automated Biomodel Selection System (BMSS) for Gene Circuit Designs.\nauthors: Yeoh, Jing Wui, Kai Boon Ivan Ng, Ai Ying Teh, JingYun Zhang, Wai Kit David Chee, and Chueh Loo Poh.\njournal: ACS synthetic biology 

If you are not sure what the exact system type is, use the ```search_database``` function.

In [6]:
search_results = mh.search_database('Inducible')
search_results[0]

{'id': 'bmss1',
 'system_type': 'BMSS, ConstantInduction, Inducible',
 'states': ['m', 'p'],
 'parameters': ['n', 'k_ind', 'synm', 'degm', 'synp', 'degp'],
 'inputs': ['ind'],
 'equations': ['dm = synm*(ind**n)/(ind**n + k_ind**n) - degm*m',
  'dp = synp*m - degp*p'],
 'ia': '',
 'descriptions': {'Description': 'This model describes a simple gene expression for mRNA and protein driven by an inducible promoter with constant induction.',
  'Definition of states': '\nm: mRNA\np: protein',
  'Definition of parameters': '\nk_ind: half-activation constant\nsynm: mRNA synthesis rate\ndegm: mRNA degradation rate\nsynp: protein synthesis rate\ndegp: protein degradation rate/dilution rate\nn: hil coefficient',
  'Definition of inputs': '\nind: inducer level',
  'Reference': '\ntitle: An Automated Biomodel Selection System (BMSS) for Gene Circuit Designs.\nauthors: Yeoh, Jing Wui, Kai Boon Ivan Ng, Ai Ying Teh, JingYun Zhang, Wai Kit David Chee, and Chueh Loo Poh.\njournal: ACS synthetic biology 

To add a model to the database, use ```config_to_database``` or ```add_to_database```. If the core model already exists in the database, a dialog will be created to ask if you want to overwrite the existing one. Set ```dialog=False``` to overwrite without being asked.

In [7]:
system_type = mh.config_to_database(filename, dialog=False)

Modified model usr47 to UBase


Deactivate the model with the ```delete``` function.

In [8]:
#It won't be searchable after this.
mh.delete(system_type)

This can be undone using ```restore```

In [9]:
mh.restore(system_type)

The database can be exported as a pandas DataFrame by using ```to_df```. This will include both active and inactive models.

In [10]:
df = mh.to_df()

### Importing Template Settings into Python
Working with template settings is similar to that with core models. However, we now use the settings_handler module instead.

In [11]:
import BMSS.models.settings_handler as sh

Connected to MBase_settings, UBase_settings


In [12]:
sh.from_config(filename)

[{'system_type': 'my_model',
  'settings_name': 'my_settings',
  'units': {'k_ind': '% Arabinose',
   'synm': 'M/min',
   'degm': '1/min',
   'synp': 'M/min',
   'degp': '1/min',
   'ind': '% Arabinose'},
  'parameters':    k_ind     synm   degm  synp   degp  ind
  0   0.01  0.00001  0.015  0.01  0.012    0,
  'init':          0         1
  1  0.00000  0.000000
  2  0.00005  0.000005,
  'priors': {'degm': array([0.015, 0.05 ]), 'degp': array([0.012, 0.04 ])},
  'parameter_bounds': {'k_ind': [0.001, 1],
   'synm': [1e-06, 0.0001],
   'degm': [0.01, 0.5],
   'synp': [0.001, 1],
   'degp': [0.001, 0.3],
   'ind': [0, 1]},
  'tspan': [array([  0.,  20.,  40.,  60.,  80., 100., 120., 140., 160., 180., 200.,
          220., 240., 260., 280., 300., 320., 340., 360., 380., 400., 420.,
          440., 460., 480., 500., 520., 540., 560., 580., 600.])],
  'fixed_parameters': [],
  'solver_args': {'rtol': 1.49012e-08,
   'atol': 1.49012e-08,
   'tcrit': [],
   'h0': 0.0,
   'hmax': 0.0,
   'hmin':

In [13]:
system_types_settings_names = sh.config_to_database(filename)


Added settings my_settings


Note that you cannot add a settings template to the database if the associated core model has not been added yet.